In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
import plotly.express as px
import matplotlib.pyplot as plt
pd.set_option('display.float_format', lambda x: '%.5f' % x)

from scipy.stats import shapiro, kstest, normaltest
from statsmodels.stats.diagnostic import lilliefors

from tqdm import tqdm
import glob
import sys  
sys.path.insert(0, '../src')

# importing user-defined functions from udf_eda.py
import udf_eda as udf
import udf_timeseries as udf_ts

In [2]:
file_names = glob.glob('../../../data/input/11_Dataset/**/ODP *.xlsx')

def Filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]
      
# Driver code
substr = ['1695', '1700', '1776', '1779']
file_names = Filter(file_names, substr)
len(file_names)

4

In [3]:
df = udf.read_bind(file_names)

# if any blank columns are created by accident in a spreadsheet software, which wouldn't have any column name, we remove such columns
df = df[df.columns.drop(list(df.filter(regex='Unnamed:')))]
print("How many NaN values exist in the data: ", df.isna().sum().sum())
print("Shape of the data: ",df.shape)

100%|██████████| 4/4 [00:12<00:00,  3.14s/it]

The following batches have incompatible data:  []
# of batches read:  4
Missing batches, if any: set()
How many NaN values exist in the data:  0
Shape of the data:  (5149, 265)


In [4]:
# Extracting the Resa variable from produzione_CStOA_2021_ed12.xlsx
tdf = pd.read_excel('../../../data/input/11_Dataset/produzione_CStOA_2021_ed12.xlsx', sheet_name="dati-produzione", header=1)
tdf = tdf[['O.D.P.','Resa']]
tdf.dropna(axis=0, how='any', inplace=True)
tdf['O.D.P.'] =tdf['O.D.P.'].astype(int).astype(str).str[-4:]
tdf.columns = ['id', 'result']
tdf.result = round(tdf.result,3)
df = tdf.merge(df, how='inner')

tdf = df.groupby(['id'])['timeseries'].agg(['min', 'max']).reset_index()
tdf.columns = ['id', 'start_date', 'end_date']
tdf['processing_time_mins'] = ((tdf['end_date'] - tdf['start_date'])/pd.Timedelta(minutes = 1))+1
df = tdf.merge(df, how = 'right')
df.insert(5, 'timestamp_index', df.groupby('id').cumcount())

In [5]:
df[['id', 'start_date', 'end_date', 'processing_time_mins', 'result']].drop_duplicates().reset_index(drop =True)
df.insert(6, 'progress_perc', round(((df.timestamp_index / df.processing_time_mins)*100),1).astype(int))

In [43]:
df = df.groupby(['id', 'start_date', 'end_date', 'processing_time_mins', 'result', 'progress_perc']
            )[df.columns[8:].tolist()].mean().reset_index()

In [45]:
df[['id', 'processing_time_mins', 'result']].drop_duplicates()

,id,processing_time_mins,result
0,1695,1135.00000,0.78600
100,1700,1251.00000,0.78700
200,1776,1228.00000,0.68900
300,1779,1535.00000,0.69000


In [46]:
df.head()

,id,start_date,end_date,processing_time_mins,result,progress_perc,101LI636,101WI610,306LI606,101AI635,...,108PI659,108PI662,108PI663,108FI653,108FI657,108FI665,108FI669,108FI673,108FI677,108FI681
0,1695,2021-09-01 17:25:00,2021-09-02 12:19:00,1135.00000,0.78600,0,84648.46307,63925.23260,0.00000,5.22869,...,5.21523,7.06779,5.19159,2.43097,2.81364,1.82673,2.33151,2.18256,2.37738,0.09772
1,1695,2021-09-01 17:25:00,2021-09-02 12:19:00,1135.00000,0.78600,1,82729.08398,63901.65299,0.00000,5.23039,...,7.51377,9.57462,7.49170,2.96621,3.28729,2.16760,2.54418,2.23021,1.98266,0.09899
2,1695,2021-09-01 17:25:00,2021-09-02 12:19:00,1135.00000,0.78600,2,80690.62855,63867.00426,0.00000,5.23300,...,9.01092,11.06981,8.98802,3.16051,3.48656,2.27925,2.54613,2.04333,1.56678,0.09803
3,1695,2021-09-01 17:25:00,2021-09-02 12:19:00,1135.00000,0.78600,3,79002.35938,63859.61506,0.00000,5.23445,...,10.24540,12.31010,10.21891,3.35724,3.64033,2.37530,2.53137,1.87282,1.27289,0.09761
4,1695,2021-09-01 17:25:00,2021-09-02 12:19:00,1135.00000,0.78600,4,77222.20703,63846.13379,0.00000,5.23590,...,11.20597,13.26363,11.18115,3.45630,3.73547,2.44023,2.49976,1.73539,1.09459,0.09966


In [47]:
df.shape

(400, 269)

In [48]:
df = df.astype({'id': 'int32'})
udf.describe(df).T


,count,mean,min,25%,50%,75%,max,std,dtype,size,perc_null
id,400.00000,1737.50000,1695.00000,1698.75000,1738.00000,1776.75000,1779.00000,40.10325,int32,400,0.00000
start_date,400,2021-09-16 13:45:30,2021-09-01 17:25:00,2021-09-06 09:13:45,2021-09-17 00:31:00,2021-09-27 05:02:45,2021-09-30 12:35:00,NaN,datetime64[ns],400,0.00000
end_date,400,2021-09-17 11:11:45,2021-09-02 12:19:00,2021-09-07 05:34:45,2021-09-17 21:09:30,2021-09-28 02:46:30,2021-10-01 14:09:00,NaN,datetime64[ns],400,0.00000
processing_time_mins,400.00000,1287.25000,1135.00000,1204.75000,1239.50000,1322.00000,1535.00000,149.67362,float64,400,0.00000
result,400.00000,0.73800,0.68900,0.68975,0.73800,0.78625,0.78700,0.04856,float64,400,0.00000
...,...,...,...,...,...,...,...,...,...,...,...
108FI665,400.00000,1.43173,0.07279,0.07950,1.41299,2.82486,3.04441,1.34129,float64,400,0.00000
108FI669,400.00000,1.27824,0.07106,0.07971,1.55268,2.52262,2.94332,1.18721,float64,400,0.00000
108FI673,400.00000,0.86518,0.09380,0.10321,1.13564,1.58796,2.75211,0.75583,float64,400,0.00000
108FI677,400.00000,0.56783,0.09392,0.10285,0.75711,0.92943,2.49642,0.48976,float64,400,0.00000


Let's perform some statistical tests between variables of each batch, the following are a few points to make note of before selecting which tests are are applicable based on the data, keep in mind we are testing between batches:
1. The samples are independent, the reason being, the values in one sample reveal no information about those of the other sample, then the samples are independent.
2. An unpaired t-test is used to compare the mean between two independent groups. You use an unpaired t-test when you are comparing two separate groups with equal variance otherwise a Welch’s test should be used.
    - To test if the variance of two groups are equal we use an F-test. This test can be a two-tailed test or a one-tailed test. The two-tailed version tests against the alternative that the variances are not equal. 
3. 

In [49]:
fdf = df.select_dtypes(include=['int32', 'int64', 'float32', 'float64'])

In [50]:
fdf

,id,processing_time_mins,result,progress_perc,101LI636,101WI610,306LI606,101AI635,101AI605,101TI607,...,108PI659,108PI662,108PI663,108FI653,108FI657,108FI665,108FI669,108FI673,108FI677,108FI681
0,1695,1135.00000,0.78600,0,84648.46307,63925.23260,0.00000,5.22869,5.48075,17.46890,...,5.21523,7.06779,5.19159,2.43097,2.81364,1.82673,2.33151,2.18256,2.37738,0.09772
1,1695,1135.00000,0.78600,1,82729.08398,63901.65299,0.00000,5.23039,5.48824,16.96575,...,7.51377,9.57462,7.49170,2.96621,3.28729,2.16760,2.54418,2.23021,1.98266,0.09899
2,1695,1135.00000,0.78600,2,80690.62855,63867.00426,0.00000,5.23300,5.49428,16.53463,...,9.01092,11.06981,8.98802,3.16051,3.48656,2.27925,2.54613,2.04333,1.56678,0.09803
3,1695,1135.00000,0.78600,3,79002.35938,63859.61506,0.00000,5.23445,5.50021,16.14001,...,10.24540,12.31010,10.21891,3.35724,3.64033,2.37530,2.53137,1.87282,1.27289,0.09761
4,1695,1135.00000,0.78600,4,77222.20703,63846.13379,0.00000,5.23590,5.50630,15.73136,...,11.20597,13.26363,11.18115,3.45630,3.73547,2.44023,2.49976,1.73539,1.09459,0.09966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1779,1535.00000,0.69000,95,2911.63674,67723.27630,82418.10781,3.80115,5.61089,12.74753,...,0.15928,-0.01093,0.12537,0.09798,0.09342,0.08042,0.08252,0.10359,0.10260,0.09354
396,1779,1535.00000,0.69000,96,2930.05229,76803.15186,84163.29297,3.80816,5.59587,13.20936,...,0.15934,-0.01273,0.12314,0.09667,0.09326,0.07822,0.08047,0.10437,0.10380,0.09543
397,1779,1535.00000,0.69000,97,10968.15033,90134.44687,90535.09427,5.21199,5.58316,13.59335,...,0.15778,-0.01232,0.12477,0.08984,0.09464,0.08063,0.07859,0.10440,0.10337,0.09445
398,1779,1535.00000,0.69000,98,33432.85781,91249.18698,95327.93385,5.59354,5.57284,13.82100,...,0.15628,-0.01168,0.12401,0.07254,0.09390,0.08141,0.08224,0.10333,0.10342,0.09308


In [51]:
fdf_columns = fdf.columns[5:].tolist()
fdf_ids = fdf.id.unique().tolist()
data = []
for i in fdf_ids:
    for j in fdf_columns:
        # print("id: %s , col: %s" % (i,j))
        x = fdf[fdf['id'] == i][j]
        # print(f'{"normal"}: {"Not Gaussian" if normaltest(x.values,)[1]<0.05 else "Gaussian"}  {normaltest(x.values)}')
        # print(f'{"KS test"}: {"Not Gaussian" if kstest(x.values,"norm")[1]<0.05 else "Gaussian"}  {kstest(x.values,"norm")}')
        # print(f'{"shapiro"}: {"Not Gaussian" if shapiro(x.values)[1]<0.05 else "Gaussian"}  {shapiro(x.values)}')
        normal      = 0 if normaltest(x.values,)[1]<0.05 else 1
        ks          = 0 if kstest(x.values,"norm")[1]<0.05 else 1
        lilli       = 0 if lilliefors(x.values)[1]<0.05 else 1
        shap        = 0 if shapiro(x.values)[1]<0.05 else 1
        cols        = ['id', 'variable', 'normaltest', 'kstest', 'lilliefors', 'shapiro']
        values      = [i, j, normal, ks, lilli, shap]
        zipped      = zip(cols, values)
        dictionary  = dict(zipped)
        data.append(dictionary)
norm_test_results = pd.DataFrame(data)

/Users/nageshk/opt/anaconda3/envs/dsenv/lib/python3.8/site-packages/statsmodels/stats/_lilliefors.py:279: RuntimeWarning: invalid value encountered in true_divide
  z = (x - x.mean()) / x.std(ddof=1)
/Users/nageshk/opt/anaconda3/envs/dsenv/lib/python3.8/site-packages/scipy/stats/morestats.py:1757: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


In [54]:
myseries = norm_test_results[(norm_test_results['shapiro'] == 1)]# | (norm_test_results['lilliefors'] == 1)]
myseries = myseries.groupby('variable').id.nunique()
myseries = myseries[myseries == 4].index.tolist()
print('The following are the variables which follow gaussian distribution in all four batches: \n', myseries)

# norm_test_results[norm_test_results['lilliefors'] == 1]

The following are the variables which follow gaussian distribution in all four batches: 
 ['108FI681', '118AI641', '118FI913', '158PIC678_823']


Now lets check if the variance is equal or not and use unpaired t-test or Welch’s test accordingly

In [55]:
import scipy
def f_test(x, y):
    x = np.array(x)
    y = np.array(y)
    f = np.var(x, ddof=1)/np.var(y, ddof=1) #calculate F test statistic 
    dfn = x.size-1 #define degrees of freedom numerator 
    dfd = y.size-1 #define degrees of freedom denominator 
    p = 1-scipy.stats.f.cdf(f, dfn, dfd) #find p-value of F test statistic 
    return f, p

for i in fdf_ids:
    for j in fdf_ids:
        for k in myseries:
            if i == j:
                continue
            else:
                x = fdf[fdf['id'] == i][k]
                y = fdf[fdf['id'] == j][k]
                F, p = f_test(x,y)
                print("F-test for variable %s between batches %s and %s is %s"% (k, i, j, p))
                flag = False if p<0.05 else True
                print("unpaired t-test for the same batches is: ", scipy.stats.ttest_ind(i,j,equal_var=flag))

F-test for variable 108FI681 between batches 1695 and 1700 is 0.1881239837878138
unpaired t-test for the same batches is:  Ttest_indResult(statistic=nan, pvalue=nan)
F-test for variable 118AI641 between batches 1695 and 1700 is 0.9999494736536803
unpaired t-test for the same batches is:  Ttest_indResult(statistic=nan, pvalue=nan)
F-test for variable 118FI913 between batches 1695 and 1700 is nan
unpaired t-test for the same batches is:  Ttest_indResult(statistic=nan, pvalue=nan)
F-test for variable 158PIC678_823 between batches 1695 and 1700 is nan
unpaired t-test for the same batches is:  Ttest_indResult(statistic=nan, pvalue=nan)
F-test for variable 108FI681 between batches 1695 and 1776 is 0.5590155346817018
unpaired t-test for the same batches is:  Ttest_indResult(statistic=nan, pvalue=nan)
F-test for variable 118AI641 between batches 1695 and 1776 is 5.292634441822486e-09
unpaired t-test for the same batches is:  Ttest_indResult(statistic=nan, pvalue=nan)
F-test for variable 118FI9

/var/folders/6s/f25fqpzn1y55r8yt0d82xjlr0000gn/T/ipykernel_16096/2347305046.py:5: RuntimeWarning: invalid value encountered in double_scalars
  f = np.var(x, ddof=1)/np.var(y, ddof=1) #calculate F test statistic


In the above output, two key highlights are:
- F-test for variable 108FI681 between batches 1695 and 1779 is (1.1241300058307309, 0.01697610895763868)
- F-test for variable 108FI681 between batches 1700 and 1779 is (1.11558008978606, 0.02095289512189924)

The above two cases reject Null hypothesis, which means that both of them do not have equal variance, therefore we should use Welch test, however when the batches are interchanged the results of the test differ

- F-test for variable 108FI681 between batches 1779 and 1695 is (0.8895768236886454, 0.9830238910423632)
- F-test for variable 108FI681 between batches 1779 and 1700 is (0.8963946283693309, 0.9790471048780995)

Let's use unpaired t-test now and check the results: